<a href="https://colab.research.google.com/github/mdsozon24/Data_Collection/blob/main/Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fake_useragent

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import re
from fake_useragent import UserAgent
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import time

# Download NLTK stopwords and punkt
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
bangla_dataset = pd.read_csv('/content/drive/MyDrive/Datasets/BanglaNews8.csv')

In [ ]:
bangla_dataset

,Category,Text,categoryId,Text_Length,Char_Length,Word_Count
0,politics,গ্যাটকো দুর্নীতি মামলায় বিএনপি চেয়ারপারসন খালে...,7,1646,1646,237
1,economy,সৈয়দ এম ওমর তৈয়ব প্রাইম ব্যাংক উপব্যবস্থাপনা প...,4,466,466,57
2,politics,প্রধান নির্বাচন কমিশন সিইসি কেএম নূরুল হুদ উদ্...,7,2976,2976,467
3,politics,আওয়ামী লীগ সম্পাদক সড়ক পরিবহন সেতুমন্ত্রী ওব...,7,2615,2615,390
4,politics,সোহেল তাজ আস দরজায় রেডি চলতি বছর এপ্রিলে খবরে ...,7,1459,1459,234
...,...,...,...,...,...,...
118399,sports,বরাবর এশিয়া কাপ চারদেশীয় প্রতিযোগিতা এশিয় সবচ ...,1,2072,2072,296
118400,sports,নাহ ভুল করেননি এবি ডি ভিলিয়ার্স টসে জিত ফিল্ডি...,1,1247,1247,198
118401,technology,পরীক্ষামূলকভাবে ভিওআইপি সেবা অপরিচিত নম্বর তথ্...,3,758,758,117
118402,sports,ভারত সিরিজ বিরতি সামান্য জুন বাংলাদেশ পা রাখ ...,1,1941,1941,295


In [ ]:
urls = [
    "https://www.jugantor.com/","https://www.prothomalo.com/","https://www.bd-pratidin.com/","https://www.ittefaq.com.bd/",
       "https://www.kalerkantho.com/","https://dailynayadiganta.com/","https://www.amarsangbad.com/","https://www.protidinersangbad.com/",
       "https://dailysangram.com/","https://mzamin.com/","https://dainikamadershomoy.com/","https://www.bonikbarta.com/",
       "https://samakal.com/","https://dailyjanakantha.com/","https://www.jaijaidinbd.com/","https://bhorerkagoj.com/",
       "https://dailyinqilab.com/","https://sangbad.net.bd/","https://suprobhat.com/","https://www.bd-journal.com/","https://www.alokitobangladesh.com/",
       "https://www.ajkerbazzar.com/","https://amaderorthoneeti.com/","https://bangladeshpost.net/","https://sorejominbarta.com/",
       "https://khoborpatrabd.com/","https://www.dailyvorerpata.com/","https://shomoyeralo.com/","https://www.kalbela.com/","https://sharebiz.net/",
       "https://www.kholakagojbd.com/","https://gonokantho.com/","https://www.thedailystar.net/","https://observerbd.com/",
       "https://thefinancialexpress.com.bd/","https://www.deshrupantor.com/","https://www.bangladesherkhabor.net/","https://bd-bulletin.com/",
       "https://www.tbsnews.net/","https://www.dhakatribune.com/","https://www.ajkerpatrika.com/","https://www.dainikbangla.com.bd/",
       "https://bangla.bdnews24.com/",
    "https://www.banglanews24.com/",
    "https://www.jagonews24.com/",
    "https://www.priyo.com/",
    "https://dainikamadershomoy.com/",
    "https://www.bd24live.com/",
    "https://www.dhakapost.com/",
    "https://www.banglatribune.com/",
    "https://www.bdnews24.com/",
    "https://unb.com.bd/",
    "https://amar-desh24.com/",
    "https://sarabangla.net/",
    "https://thedhakatimes.com/",
    "https://dmpnews.org/",
    "https://www.thereport24.com/",
    "https://somoyerkonthosor.com/",
    "https://sharenews24.com/",
    "https://sunbd24.com/",
    "https://orthosongbad.com/",
    "https://ajkersottasangbad24.com/",
    "https://thedhakapost.com/",
    "https://www.onenewsbd.com/",
    "https://deshnews.net/",
    "https://deshersangbad.com/",
    "https://lastnewsbd.com/",
    "https://www.bangladaily24.com/",
    "https://www.protikhon.com/",
    "https://bartabangla.com/",
    "https://bdtype.com/",
    "https://inews.zoombangla.com/",
    "https://khoborprotidin24.com/",
    "https://bdcrictime.com/",
    "https://www.bbc24news.com/",
    "https://www.latestbdnews.com/",
    "https://bartabazar.com/",
    "https://www.dhakaprotidin.com/",
    "https://eibela.com/",
    "https://techshohor.com/",
    "https://techzoom.tv/"
    ]

In [ ]:
session = requests.Session()

In [ ]:
df = []

In [ ]:
ua = UserAgent()

retry_strategy = Retry(
    total=3,  # Maximum number of retries
    backoff_factor=1,  # Exponential backoff factor (1s, 2s, 4s delays)
    status_forcelist=[429, 500, 502, 503, 504],  # HTTP statuses to retry on
    allowed_methods=["HEAD", "GET", "OPTIONS"] # HTTP methods to retry
)

In [ ]:
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("http://", adapter)
session.mount("https://", adapter)

In [ ]:
for url in urls:
    try:
        headers = {'User-Agent': ua.random, 'Accept-Language': 'en,bn'} # Generate and add a random user agent and accept language
        r = session.get(url, timeout=10, headers=headers)
        r.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)

        soup = BeautifulSoup(r.text, 'html.parser')
        data = soup.get_text()

        for line in data.splitlines():
            cleaned_line = re.sub(r'\s+', ' ', line.strip())
            if cleaned_line and len(cleaned_line) > 20:
                df.append(cleaned_line)

    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
    except Exception as e:
        print(f"An unexpected error occurred while processing {url}: {e}")

    time.sleep(2)

Error fetching https://www.kalerkantho.com/: 403 Client Error: Forbidden for url: https://www.kalerkantho.com/
Error fetching https://www.protidinersangbad.com/: 403 Client Error: Forbidden for url: https://www.protidinersangbad.com/
Error fetching https://dailyjanakantha.com/: 403 Client Error: Forbidden for url: https://dailyjanakantha.com/
Error fetching https://www.jaijaidinbd.com/: 403 Client Error: Forbidden for url: https://www.jaijaidinbd.com/
Error fetching https://www.bd-journal.com/: 403 Client Error: Forbidden for url: https://www.bd-journal.com/


Error fetching https://www.ajkerbazzar.com/: HTTPSConnectionPool(host='www.ajkerbazzar.com', port=443): Max retries exceeded with url: / (Caused by ReadTimeoutError("HTTPSConnectionPool(host='www.ajkerbazzar.com', port=443): Read timed out. (read timeout=10)"))


Error fetching https://khoborprotidin24.com/: 403 Client Error: Forbidden for url: https://khoborprotidin24.com/
Error fetching https://techshohor.com/: 403 Client Error: Forbidden for url: https://techshohor.com/


In [ ]:
df

['Jugantor: Bangla Newspaper | Bangla News | ব্রেকিং নিউজ',
 'স্বরাষ্ট্র উপদেষ্টা প্রতিবেশী দেশ ও ফ্যাসিস্টের সহযোগীরা গুজব সৃষ্টির চেষ্টা করবে',
 'স্বরাষ্ট্র উপদেষ্টা লেফটেন্যান্ট জেনারেল (অব.) মো. জাহাঙ্গীর আলম চৌধুরী বলেছেন, দুর্গাপূজা ভালোভাবে সম্পন্ন হবে। এ নিয়ে কোনো শঙ্কা নেই। তবে প্রতিবেশী দেশ ও ফ্যা ...',
 '২৯ সেপ্টেম্বর ২০২৫, ১২:৫৪ এএম',
 'জামায়াত আমিরের সঙ্গে স্প্যানিশ রাষ্ট্রদূতের সাক্ষাৎ, আলোচনা হলো যেসব বিষয়ে',
 'রিমান্ডের আদেশ শুনে কাঠগড়ায় ঢলে পড়েন এনায়েত করিম',
 'ভিসা বাতিল যুক্তরাষ্ট্রের বিরুদ্ধে পালটা অভিযোগ কলম্বিয়ার প্রেসিডেন্টের',
 'নীলক্ষেতে ডাকসু নির্বাচনের ব্যালট ছাপানোর অভিযোগ, যা বললেন ঢাবি উপাচার্য',
 'ভোটের সঙ্গে সম্পৃক্তদেরও ভোট দেওয়ার ব্যবস্থা করা হয়েছে: সিইসি',
 'পাকিস্তানে সেনা অভিযানে বাংলাদেশি যুবক নিহত',
 'পাকিস্তানকে ‘সন্ত্রাসবাদের কেন্দ্র’ বললেন জয়শঙ্কর',
 'সৌহার্দ্য-সম্প্রীতির বার্তা সর্বত্র ছড়িয়ে দিন: তারেক রহমান',
 'আগামী নির্বাচন হবে সব ধরনের ধারণাকে ঘুরিয়ে দেওয়ার নির্বাচন: এনসিপি নেত্রী',
 'ফাইনালের আগে নতুন বিতর্কের জন্ম দিল ভারত, যা বললেন পা

In [ ]:
df = pd.DataFrame(df, columns=['Text'])
df.head()

,Text
0,Jugantor: Bangla Newspaper | Bangla News | ব্র...
1,স্বরাষ্ট্র উপদেষ্টা প্রতিবেশী দেশ ও ফ্যাসিস্টে...
2,স্বরাষ্ট্র উপদেষ্টা লেফটেন্যান্ট জেনারেল (অব.)...
3,"২৯ সেপ্টেম্বর ২০২৫, ১২:৫৪ এএম"
4,জামায়াত আমিরের সঙ্গে স্প্যানিশ রাষ্ট্রদূতের সা...
